## Generate the observations

In [1]:
import numpy as np
import openturns as ot

In [2]:
ot.RandomGenerator.SetSeed(0)

In [3]:
def modelChaboche(X):
    strain,R,C,gamma = X
    stress = R + C*(1-np.exp(-gamma*strain))
    return [stress]

In [4]:
g = ot.PythonFunction(4, 1, modelChaboche) 

In [5]:
Strain = ot.Uniform(0,0.07)
unknownR = 750
unknownC = 2750
unknownGamma = 10
R = ot.Dirac(unknownR)
C = ot.Dirac(unknownC)
Gamma = ot.Dirac(unknownGamma)

Strain.setDescription(["Strain"])
R.setDescription(["R"])
C.setDescription(["C"])
Gamma.setDescription(["Gamma"])

Create the joint input distribution function.

In [6]:
inputRandomVector = ot.ComposedDistribution([Strain, R, C, Gamma])

Create the Monte-Carlo sample.

In [7]:
size = 5
inputSample = inputRandomVector.getSample(size)
outputStress = g(inputSample)
outputStress

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=1 description=[y0] data=[[1730.52],[2017.64],[998.458],[811.861],[1343.12]]

Generate observation noise.

In [8]:
stressObservationNoiseSigma = 40 # (Pa)
noiseSigma = ot.Normal(0.,stressObservationNoiseSigma)
sampleNoiseH = noiseSigma.getSample(size)
outputObservations = outputStress + sampleNoiseH

In [9]:
inputObservations = inputSample[:,0]

## Set the calibration parameters

Define the value of the reference values of the $\theta$ parameter. In the bayesian framework, this is called the mean of the *prior* gaussian distribution. In the data assimilation framework, this is called the *background*.

In [10]:
R = 700 # Exact : 750
C = 2500 # Exact : 2750
Gamma = 8. # Exact : 10
candidate = ot.Point([R,C,Gamma])

The following statement create the calibrated function from the model. The calibrated parameters Ks, Zv, Zm are at indices 1, 2, 3 in the inputs arguments of the model.

In [11]:
calibratedIndices = [1,2,3]
model = ot.ParametricFunction(g, calibratedIndices, candidate)

## Gaussian linear calibration

The standard deviation of the observations.

In [12]:
sigmaStress = 10. # (Pa)

Define the covariance matrix of the output Y of the model.

In [13]:
errorCovariance = ot.CovarianceMatrix(1)
errorCovariance[0,0] = sigmaStress**2
errorCovariance

class=CovarianceMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[100]

Defined the covariance matrix of the parameters $\theta$ to calibrate.

In [14]:
sigmaR = 0.1 * R
sigmaC = 0.1 * C
sigmaGamma = 0.1 * Gamma

In [15]:
parameterCovariance = ot.CovarianceMatrix(3)
parameterCovariance[0,0] = sigmaR**2
parameterCovariance[1,1] = sigmaC**2
parameterCovariance[2,2] = sigmaGamma**2
parameterCovariance

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[4900,0,0,0,62500,0,0,0,0.64]

## Calibration

The `GaussianLinearCalibration` class performs the gaussian linear calibration by linearizing the model in the neighbourhood of the prior.

In [16]:
algo = ot.GaussianLinearCalibration(model, inputObservations, outputObservations, candidate, parameterCovariance, errorCovariance)

The `run` method computes the solution of the problem.

In [17]:
algo.run()

In [18]:
calibrationResult = algo.getResult()

## Analysis of the results

The `getParameterMAP` method returns the maximum of the posterior distribution of $\theta$.

In [19]:
thetaStarOT = calibrationResult.getParameterMAP()
thetaStarOT

class=Point name=Unnamed dimension=3 values=[739.455,2746.96,10.6293]

We can compute a 95% confidence interval of the parameter $\theta^\star$. 

In [20]:
thetaPosterior = calibrationResult.getParameterPosterior()
thetaPosterior

class=Normal name=Normal dimension=3 mean=class=Point name=Unnamed dimension=3 values=[739.455,2746.96,10.6293] sigma=class=Point name=Unnamed dimension=3 values=[8.08963,143.471,0.593968] correlationMatrix=class=CorrelationMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[1,0.1796,-0.357051,0.1796,1,-0.974716,-0.357051,-0.974716,1]

In [21]:
covarianceThetaStarOT = thetaPosterior.getCovariance()
covarianceThetaStarOT

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[65.4421,208.449,-1.71562,208.449,20584,-83.0627,-1.71562,-83.0627,0.352798]

## Calibration based on Kalman matrix

In [22]:
parameterDimension = candidate.getDimension()

In [23]:
model.setParameter(candidate)

In [24]:
modelObservations = model(inputObservations)
modelObservations

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=1 description=[y0] data=[[1443.08],[1675.11],[882.391],[745.092],[1141.58]]

In [25]:
transposedGradientObservations = ot.Matrix(parameterDimension,size)
for i in range(size):
    grad = model.parameterGradient(inputObservations[i])
    transposedGradientObservations[:,i] = grad
transposedGradientObservations

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=5 values=[1,0.297234,77.4648,1,0.390046,94.2324,1,0.0729565,21.9462,1,0.0180369,5.58539,1,0.176632,50.0072]

In [26]:
gradientObservations = transposedGradientObservations.transpose()
gradientObservations

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=3 values=[1,1,1,1,1,0.297234,0.390046,0.0729565,0.0180369,0.176632,77.4648,94.2324,21.9462,5.58539,50.0072]

In [27]:
deltay = outputObservations - modelObservations
deltay = deltay.asPoint()
deltay[0:5]

class=Point name=Unnamed dimension=5 values=[364.348,343.228,89.6333,55.801,236.21]

Compute inverses of B and R.

In [28]:
B = ot.CovarianceMatrix(parameterCovariance)
IB = ot.IdentityMatrix(parameterDimension)
invB = B.solveLinearSystem(IB)

In [29]:
R = ot.CovarianceMatrix(size)
for i in range(size):
    R[i,i] = errorCovariance[0,0]
R[0:5,0:5]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=5 values=[100,0,0,0,0,0,100,0,0,0,0,0,100,0,0,0,0,0,100,0,0,0,0,0,100]

In [30]:
IR = ot.IdentityMatrix(size)
invR = R.solveLinearSystem(IR)

Calcule $A^{-1} = B^{-1} + J^T R^{-1} J = B^{-1} + J^T (J^T R^{-1})^T$.

Soit $C =J^T R^{-1}$. 

Cela implique $A^{-1} = B^{-1} + J^T C^T$.

In [31]:
C = gradientObservations.transpose() * invR

In [32]:
invA = invB + C * gradientObservations
invA

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[0.0502041,0.00954905,2.49236,0.00954905,0.0027893,0.703148,2.49236,0.703148,180.504]

Calcule $K = A J^T R^{-1}$ en tant que solution de l'équation matricielle $A^{-1}K = C$.

In [33]:
K = invA.solveLinearSystem(C)
K

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=5 values=[-0.0550017,-1.07737,0.00924793,-0.149206,4.09934,-0.00868816,0.429984,-1.12731,-0.000329889,0.596195,1.15782,-0.012433,0.164673,-3.09502,0.0125532]

Calcule $\hat{\theta} = \mu  + K (y - H(\mu ))$.

In [34]:
thetaStar = candidate + K * deltay
thetaStar

class=Point name=Unnamed dimension=3 values=[739.455,2746.96,10.6293]

In [35]:
thetaStarOT - thetaStar

class=Point name=Unnamed dimension=3 values=[0,0,0]

Calcule la matrice $A$.

In [36]:
IA = ot.IdentityMatrix(parameterDimension)
A = invA.solveLinearSystem(IA)
A

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[65.4421,208.449,-1.71562,208.449,20584,-83.0627,-1.71562,-83.0627,0.352798]

In [37]:
A - covarianceThetaStarOT

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[-2.62901e-12,-1.58309e-11,9.72555e-14,-1.83888e-10,-6.43922e-10,5.01643e-12,7.69385e-13,2.84217e-12,-2.15938e-14]

Manifestement, le résultat est faux.

Un autre moyen de calculer la matrice de covariance.

In [38]:
L = IB - K * gradientObservations

In [39]:
covarianceThetaStar = K * R * K.transpose() + L * B * L.transpose()
covarianceThetaStar

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[65.4421,208.449,-1.71562,208.449,20584,-83.0627,-1.71562,-83.0627,0.352798]

In [40]:
(covarianceThetaStar - covarianceThetaStarOT)/1.e15

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[0,2.84217e-29,0,5.68434e-29,0,0,2.22045e-31,0,-5.55112e-32]

On observe que l'on obtient pas du tout les memes résultats... La cause vraisemblable de ce problème est que la matrice est très mal conditionnée.

In [41]:
np.log10(np.linalg.cond(invA))

6.5701557871206235

## Par utilisation de la décomposition de Cholesky

Soit $L_B \in \mathbb{R}^{p\times p}$ le facteur de Cholesky de la matrice de covariance $B$ :

$$
B = L_B L_B^T
$$

où $L_B$ est une matrice triangulaire inférieure. 

De meme, soit $L_R \in \mathbb{R}^{n\times n}$ le facteur de Cholesky de la matrice de covariance $R$ :

$$
R = L_R L_R^T
$$

où $L_R$ est une matrice triangulaire inférieure.

In [42]:
LR = R.computeCholesky()
LR[0:5,0:5]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=5 values=[10,0,0,0,0,0,10,0,0,0,0,0,10,0,0,0,0,0,10,0,0,0,0,0,10]

In [43]:
LB = B.computeCholesky()
LB

class=TriangularMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[70,0,0,0,250,0,0,0,0.8]

On calcule $E = L_R^{-1} J L_B$.

In [44]:
IR = ot.IdentityMatrix(size)
invLR = LR.solveLinearSystem(IR)
invLR[0:5,0:5]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=5 values=[0.1,0,0,0,0,0,0.1,0,0,0,0,0,0.1,0,0,0,0,0,0.1,0,0,0,0,0,0.1]

In [45]:
E = invLR * gradientObservations * LB
E [0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=3 values=[7,7,7,7,7,7.43084,9.75114,1.82391,0.450922,4.4158,6.19718,7.53859,1.7557,0.446831,4.00058]